### 문제 2-4 : 여행 계획 분석기

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:4])

gsk_


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

response_schemas = [
    ResponseSchema(name="destination", description="여행한 장소 또는 여행할 목적지"),
    ResponseSchema(name="duration", description="여행 기간 (예: '2박 3일', '5일간')"),
    ResponseSchema(name="budget", description="여행 예산 또는 사용한 비용 (예: '30만원', '100달러')"),
    ResponseSchema(name="rating", description="여행의 추천도, 1~5점 중 하나의 숫자"),
    ResponseSchema(name="activities", description="여행 중 주요 활동들을 리스트 형태로 출력")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-CGunLKE6-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행한 장소 또는 여행할 목적지
	"duration": string  // 여행 기간 (예: '2박 3일', '5일간')
	"budget": string  // 여행 예산 또는 사용한 비용 (예: '30만원', '100달러')
	"rating": string  // 여행의 추천도, 1~5점 중 하나의 숫자
	"activities": string  // 여행 중 주요 활동들을 리스트 형태로 출력
}
```


In [3]:

template = """
다음 여행 후기나 여행 계획 텍스트를 분석하여
여행지, 기간, 예산, 추천도(1~5점), 주요 활동 리스트를 추출하세요.

여행 텍스트: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1", 
    model="meta-llama/llama-4-scout-17b-16e-instruct", 
    temperature=0.7
)

review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 
해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 
정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

chain = prompt | model | parser
output = chain.invoke({"review": review})

print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'activities': ['해운대 바다구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4'}
